## LND REST API example

### senario

* multi-hop payment (Alice -> Bob -> Charlie)

### setup

In [ ]:
# load libraries
import json
from threading import Thread
from base64 import b64decode
from time import sleep
from client import LndClient, BtcClient
from util import p, dump, generate_blocks

In [ ]:
# node
alice   = LndClient("lnd1", "8081")
bob     = LndClient("lnd2", "8082")
charlie = LndClient("lnd3", "8083")
bitcoin = BtcClient()

# initialize mainchain
bitcoin.generate(101)
p('block height = {}'.format(bitcoin.getblockcount()))

### 1. fund Alice, Bob

* Alice: 0.09 btc
* Charlie: 0.08 btc

In [ ]:
p('[wallet balance before funding]')
p(" alice = {}".format(alice.get('/balance/blockchain')))
p(" bob   = {}".format(  bob.get('/balance/blockchain')))

addr_a = alice.get('/newaddress', {'type': 'np2wkh'})['address']
addr_b =   bob.get('/newaddress', {'type': 'np2wkh'})['address']
bitcoin.sendtoaddress(addr_a, 0.09)
bitcoin.sendtoaddress(addr_b, 0.08)
bitcoin.generate(6)

p('[wallet balance after funding]')
p(" alice = {}".format(alice.get('/balance/blockchain')))
p(" bob   = {}".format(  bob.get('/balance/blockchain')))

### 2. connect nodes

* Alice -> Bob
* Bob   -> Charlie

In [ ]:
# Alice -> Bob
pubkey_b = bob.get('/getinfo')['identity_pubkey']
host_b = 'lnd2:9735'
alice.post('/peers', { 'addr': { 'pubkey': pubkey_b, 'host': host_b }, 'perm': True })

# Bob -> Charlie
pubkey_c = charlie.get('/getinfo')['identity_pubkey']
host_c = 'lnd3:9735'
bob.post('/peers', { 'addr': { 'pubkey': pubkey_c, 'host': host_c }, 'perm': True })

p('[identity pubkey]')
p("  bob     = {}".format(pubkey_b))
p("  charlie = {}".format(pubkey_c))

p('[peer]')
p('  alice   <-> ', end=''); dump(  alice.get('/peers'))
p('  bob     <-> ', end=''); dump(    bob.get('/peers'))
p('  charlie <-> ', end=''); dump(charlie.get('/peers'))

### 3. open the channel

In [ ]:
# Alice to Bob
point_a =  alice.post('/channels', { 
    "node_pubkey_string": pubkey_b, "local_funding_amount": "7000000", "push_sat": "0" 
})

# Bob to Charlie
point_b = bob.post('/channels', { 
    "node_pubkey_string": pubkey_c, "local_funding_amount": "6000000", "push_sat": "0"
})

# open the channel
sleep(2)
bitcoin.generate(6)

# check the channel state
p('[channels: alice]'); dump(alice.get('/channels'))
p('[channels: bob]'  ); dump(  bob.get('/channels'))

In [ ]:
p('[channel: Alice <-> Bob]')
funding_tx_id_a = b64decode(point_a['funding_txid_bytes'].encode())[::-1].hex()
output_index_a = point_a['output_index']
p('  funding tx txid   = {}'.format(funding_tx_id_a))
p('  funding tx vout n = {}'.format(output_index_a))

p('[channel: Bob <-> Charlie]')
funding_tx_id_b  = b64decode(point_b['funding_txid_bytes'].encode())[::-1].hex()
output_index_b = point_b['output_index']
p('  funding tx txid   = {}'.format(funding_tx_id_b))
p('  funding tx vout n = {}'.format(output_index_b))

### 4. create invoice

* Charlie charges Alice 40,000 satoshi

In [ ]:
# add a invoice to the invoice database
invoice = charlie.post('/invoices', { "value": "40000" })

# check the invoice
invoice_info = charlie.get('/invoice/' + b64decode(invoice['r_hash'].encode()).hex())
p('[invoice]'); dump(invoice_info)

# check the payment request
payreq = charlie.get('/payreq/' + invoice['payment_request'])
p('[payment request]'); dump(payreq)

### 5. send the payment

* Alice pays 40,000 satoshi to Charlie
* If you have the error "unable to find a path to destination", please wait a little while and then try again.

In [ ]:
# check the channel balance
p('[channel balance before paying]')
p('  alice   = {}'.format(  alice.get('/balance/channels')))
p('  charlie = {}'.format(charlie.get('/balance/channels')))

# send the payment
payment = alice.post('/channels/transactions', { 'payment_request': invoice['payment_request'] })
p('[payment]'); dump(payment)

# check the payment
# p('[payment]'); dump(alice.get('/payments'))

# wait
sleep(2)

# check the channel balance
p('[channel balance after paying]')
p('  alice   = {}'.format(  alice.get('/balance/channels')))
p('  charlie = {}'.format(charlie.get('/balance/channels')))

### 6. close the channel

In [ ]:
# check the balance
p('[channel balance]')
p(' alice   = {}'.format(  alice.get('/balance/channels')))
p(' bob     = {}'.format(    bob.get('/balance/channels')))
p(' charlie = {}'.format(charlie.get('/balance/channels')))

p('[wallet balance]')
p(' alice   = ', end=''); dump(  alice.get('/balance/blockchain')['confirmed_balance'])
p(' bob     = ', end=''); dump(    bob.get('/balance/blockchain')['confirmed_balance'])
p(' charlie = ', end=''); dump(charlie.get('/balance/blockchain')['confirmed_balance'])


p('[channel: Alice <-> Bob]')

# mine mainchain 1 block after 3 sec
Thread(target=generate_blocks, args=(bitcoin, 1, 3)).start()

# check the channel state before closing
p(' number of channels : {}'.format(len(alice.get('/channels')['channels'])))

# close the channel
res = alice.delete('/channels/' + funding_tx_id_a + '/' + str(output_index_a)).text.split("\n")[1]
closing_txid_a = json.loads(res)['result']['chan_close']['closing_txid']
p(' closing_txid = {}'.format(closing_txid_a))
sleep(5)

# check the channel state after closing
p(' number of channels : {}'.format(len(alice.get('/channels')['channels'])))


p('[channel: Bob <-> Charlie]')

# mine mainchain 1 block after 3 sec
Thread(target=generate_blocks, args=(bitcoin, 1, 3)).start()

# close the channel
res = bob.delete('/channels/' + funding_tx_id_b + '/' + str(output_index_b)).text.split("\n")[1]
closing_txid_b = json.loads(res)['result']['chan_close']['closing_txid']
p(' closing_txid = {}'.format(closing_txid_b))
sleep(5)

# check the balance
p('[channel balance]')
p(' alice   = {}'.format(  alice.get('/balance/channels')))
p(' bob     = {}'.format(    bob.get('/balance/channels')))
p(' charlie = {}'.format(charlie.get('/balance/channels')))

p('[wallet balance]')
p(' alice   = ', end=''); dump(  alice.get('/balance/blockchain')['confirmed_balance'])
p(' bob     = ', end=''); dump(    bob.get('/balance/blockchain')['confirmed_balance'])
p(' charlie = ', end=''); dump(charlie.get('/balance/blockchain')['confirmed_balance'])